In [ ]:
import cv2

In [ ]:
from datetime import datetime, timedelta

last_send_time = datetime.now()



In [ ]:
import requests

# URL of the endpoint
url = "http://localhost:8000/api/email/sendCat"

In [ ]:
from ultralytics import YOLO

In [ ]:
# Load a model
model = YOLO("yolov8m.yaml")  # build a new model from scratch
model = YOLO("yolov8m.pt")  # load a pretrained model (recommended for training)

In [ ]:
# Initialize the camera
camera = cv2.VideoCapture(0)  # 0 represents the default camera (you can change it if you have multiple cameras)

# Check if the camera is opened successfully
if not camera.isOpened():
    print("Error: Could not open camera.")
    exit()

# Capture a frame from the camera
ret, frame = camera.read()

# Check if the frame was captured successfully
if not ret:
    print("Error: Could not read frame.")
    camera.release()  # Release the camera
    exit()

# Save the captured frame as an image file
image_filename = "captured_image.jpg"
cv2.imwrite(image_filename, frame)

# Release the camera
camera.release()

print(f"Image captured and saved as {image_filename}")

In [ ]:
results = model(image_filename)  # predict on an image

In [ ]:
if len(results) > 0:
    result = results[0]
    result_cls = [result.names[i] for i in result.boxes.cls.tolist()]

    if 'cat' in result_cls:
        print("cat is here")
        
        current_time = datetime.now()
        current_hour = datetime.now().hour
        time_difference = current_time - last_send_time
        
        print("time_difference: ", time_difference)
        
        if time_difference > timedelta(hours=1):
            if 7 <= current_hour and current_hour < 20:
                try:
                    # Sending a POST request
                    response = requests.get(url)

                    # Checking the response status code
                    if response.status_code == 200:
                        print("POST request sent successfully.")
                    else:
                        print(f"Failed to send POST request. Status code: {response.status_code}")

                except Exception as e:
                    print(f"An error occurred: {e}")